In [1]:
import json
import cv2
import trt_pose.coco

with open('human_pose.json', 'r') as f:
    human_pose = json.load(f)

topology = trt_pose.coco.coco_category_to_topology(human_pose)



import trt_pose.models

num_parts = len(human_pose['keypoints'])
num_links = len(human_pose['skeleton'])

model = trt_pose.models.resnet18_baseline_att(num_parts, 2 * num_links).cuda().eval()



import torch

MODEL_WEIGHTS = 'resnet18_baseline_att_224x224_A_epoch_249.pth'

model.load_state_dict(torch.load(MODEL_WEIGHTS))



WIDTH = 224
HEIGHT = 224

data = torch.zeros((1, 3, HEIGHT, WIDTH)).cuda()


import torch2trt

# model_trt = torch2trt.torch2trt(model, [data], fp16_mode=True, max_workspace_size=1<<25)

OPTIMIZED_MODEL = 'resnet18_baseline_att_224x224_A_epoch_249_trt.pth'

# torch.save(model_trt.state_dict(), OPTIMIZED_MODEL)

from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load(OPTIMIZED_MODEL))


from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load(OPTIMIZED_MODEL))


import cv2
import torchvision.transforms as transforms
import PIL.Image

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda()
device = torch.device('cuda')


def preprocess(image):
    global device
    device = torch.device('cuda')
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device)
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]



# from trt_pose.draw_objects import DrawObjects
from pandas import DataFrame as pd
from trt_pose.draw_objects import DrawObjects
from trt_pose.parse_objects import ParseObjects


parse_objects = ParseObjects(topology)
draw_objects = DrawObjects(topology)

_______

In [2]:
# from jetcam.usb_camera import USBCamera
from jetcam.csi_camera import CSICamera
from jetcam.utils import bgr8_to_jpeg

if 'camera' in globals():
    print('Camera Rebooted')
    camera.running = False
    camera.cap.release()
#     camera.release()
# camera = USBCamera(width=WIDTH, height=HEIGHT, capture_fps=30)
#Sony IMX219 max rez: 3280 × 2464
camera = CSICamera(width=3264, height=2464, capture_width=3264, capture_height=2464, capture_fps=21)
#cameraFull = CSICamera(width=WIDTH, height=HEIGHT, capture_fps=20)
camera.running = True







GST_ARGUS: 3264 x 2464 FR = 21.000000 fps Duration = 47619048 ; Analog Gain range min 1.000000, max 10.625000; Exposure Range min 13000, max 683709000;

GST_ARGUS: 3264 x 1848 FR = 28.000001 fps Duration = 35714284 ; Analog Gain range min 1.000000, max 10.625000; Exposure Range min 13000, max 683709000;

GST_ARGUS: 1920 x 1080 FR = 29.999999 fps Duration = 33333334 ; Analog Gain range min 1.000000, max 10.625000; Exposure Range min 13000, max 683709000;

GST_ARGUS: 1640 x 1232 FR = 29.999999 fps Duration = 33333334 ; Analog Gain range min 1.000000, max 10.625000; Exposure Range min 13000, max 683709000;

GST_ARGUS: 1280 x 720 FR = 59.999999 fps Duration = 16666667 ; Analog Gain range min 1.000000, max 10.625000; Exposure Range min 13000, max 683709000;

GST_ARGUS: 1280 x 720 FR = 120.000005 fps Duration = 8333333 ; Analog Gain range min 1.000000, max 10.625000; Exposure Range min 13000, max 683709000;

Benchmark:

In [3]:
# ###### Benchmark:

# t0 = time.time()
# torch.cuda.current_stream().synchronize()
# for i in range(50):
#     y = model_trt(data)
# torch.cuda.current_stream().synchronize()
# t1 = time.time()

# print(50.0 / (t1 - t0))

# ###### :Benchmark

_______

In [4]:
# def get_keypoints(image, human_pose, topology, object_counts, objects, normalized_peaks):
# # """Get the keypoints from torch data and put into a dictionary where keys are keypoints
# # and values the x,y coordinates. The coordinates will be interpreted on the image given.

# # Args:
# #     image: cv2 image
# #     human_pose: json formatted file about the keypoints

# # Returns:
# #     dictionary: dictionary where keys are keypoints and values are the x,y coordinates
# # """

#     height = image.shape[0]
#     width = image.shape[1]
#     keypoints = {}
#     K = topology.shape[0]
#     count = int(object_counts[0])

#     for i in range(count):
#         obj = objects[0][i]
#         C = obj.shape[0]
#         for j in range(C):
#             k = int(obj[j])
#             if k >= 0:
#                 peak = normalized_peaks[0][j][k]
#                 x = round(float(peak[1]) * width)
#                 y = round(float(peak[0]) * height)
#                 keypoints[human_pose["keypoints"][j]] = (x, y)

#     return keypoints




Next, we'll create a widget which will be used to display the camera feed with visualizations.

In [5]:
import time
from time import time as time1

Cwidth  = 3280
Cheight = 2464

averageX = 0.500
averageY = 0.500

centerX = 1640
centerY = 1232
scale   = 0.8  # 0.2 max
notFound = 0
flag = 1
record = False




# rotation arm

In [6]:
# vieo record:
# filename = 'video.avi'
filename = time.strftime("%Y%m%d-%H%M%S.avi")

VIDEO_TYPE = {
    'avi': cv2.VideoWriter_fourcc(*'XVID'),
    'mp4': cv2.VideoWriter_fourcc(*'H264'),
#     'mp4': cv2.VideoWriter_fourcc(*'XVID'),
}

# def get_video_type(filename):
#     filename, ext = os.path.splitext(filename)
#     if ext in VIDEO_TYPE:
#         return  VIDEO_TYPE[ext]
#     return VIDEO_TYPE['avi']


# Rout = cv2.VideoWriter(filename, VIDEO_TYPE['avi'], 10,  frameSize=(WIDTH, HEIGHT))
# end = time1() + 120

In [7]:
def get_keypoint(humans, hnum, peaks):
    global averageX, averageY, notFound, scale
    names = [
        "nose           ",  #  -- 0
        "left_eye       ",  #  -- 1
        "right_eye      ",  #  -- 2
        "left_ear       ",  #  -- 3
        "right_ear      ",  #  -- 4
        "left_shoulder  ",  #  -- 5
        "right_shoulder ",  #  -- 6
        "left_elbow     ",  #  -- 7
        "right_elbow    ",  #  -- 8
        "left_wrist     ",  #  -- 9
        "right_wrist    ",  #  -- 10
        "left_hip       ",  #  -- 11
        "right_hip      ",  #  -- 12
        "left_knee      ",  #  -- 13
        "right_knee     ",  #  -- 14
        "left_ankle     ",  #  -- 15
        "right_ankle    ",  #  -- 16
        "neck           "   #  -- 17
    ]
        
        
    #check invalid human index
    kpoint = []
    human = humans[0][hnum]
    C = human.shape[0]
    usefulData1 = 0
    usefulData2 = 0

    for j in range(C):
        k = int(human[j])
        if k >= 0:
            peak = peaks[0][j][k]   
            peak = (j, float(peak[0]), float(peak[1])) # peak[2]:width, peak[1]:height
            kpoint.append(peak)
            
            
#             print('%d:\t%s : success [%5.3f, %5.3f]'%(j, names[j], peak[1], peak[2]))

#             !! average mid point  
            if k == 0:
                if (j == 0): 
                    averageY = peak[1]
                    averageX = peak[2]
                    notFound = 0

                #     Auto ZOOM!!!
               
                if j == 5: #sholders #L
                    usefulData1 += 1
                    dataIndex5 = peak[2]

                if j == 6: #R
                    usefulData1 += 1
                    dataIndex6 = peak[2]

                if j == 1: #eyes #L
                    usefulData2 += 1
                    dataIndex1 = peak[2]
                if j == 2: #R
                    usefulData2 += 1
                    dataIndex2 = peak[2]
   
        else:    
            peak = (j, None, None)
            kpoint.append(peak)
#             print('%d:\t%s : None'%(j,names[j]) )
            if (j == 0): 
                if notFound <= 4:
                    notFound += 1
                if notFound >= 2:
                    averageX = 0.5
                    averageY = 0.5

                    

        
    #send data if scale found:
    
#     if k == 0:
#         print("R-sholder: ", end='')
#         findAngle(kpoint[17],kpoint[6],kpoint[8])
#         print("R-elbow  : ", end='')
#         findAngle(kpoint[6],kpoint[8],kpoint[10])
#         print(" ")
#         print("L-sholder: ", end='')
#         findAngle(kpoint[17],kpoint[5],kpoint[7])
#         print("L-elbow  : ", end='')
#         findAngle(kpoint[5],kpoint[7],kpoint[9])
#         print(" ")  
#     //0753
    
    if usefulData1 == 2:
        #we get radius
        radiusX = (abs(dataIndex5 - dataIndex6)-0.2)
#         print(radiusX)
        scale += radiusX/4
#         print(scale)
        

    elif usefulData2 == 2:
        radiusX = (abs(dataIndex1 - dataIndex2)-0.1)
        scale += radiusX/4
        
        
    if scale > 0.95:
        scale = 0.95
    if scale < 0.2:
        scale = 0.2
    
    return kpoint



def scan(radX,radY):
    global centerX, centerY, notFound

    

    if centerX >= (Cwidth - radX - 1):
        centerX = 0
        
        if centerY>= (Cheight - radY - 1 ):
                notFound += 1
                centerY = 0
                
        centerY += (radY)
    centerX += (radX)
    return 0




def execute(change):
    global averageX, averageY, centerX, centerY, notFound, scale, flag, end
    clear_output()
    start = time.time()
    preimage = change['new'] # get new image 
    
    #preperocess image before logic preprocess 
    
    #!!TODO
#      - Max zoom 
#      - edge crop
#      - edge zoom
#      - maximase reson
     
#      -no object found:
#      -- slow zoom middel
#      -- zoom scan


#     radiusX = int(scale*(Cwidth /2))
    radiusX = radiusY = int(scale*(Cheight/2))
    
    
    

#     print('%5.3f, %5.3f'%(averageX-0.5, averageY-0.5))


    
#     myList.append((averageX,averageY))
    centerX += int((averageX-0.5)*radiusX)
    centerY += int((averageY-0.5)*radiusY)

#     print('%5.3f, %5.3f'%((averageX-0.5)*radiusX, (averageY-0.5)*radiusY))
#     print('%5.3f, %5.3f'%(centerX, centerY))
    
    
   
      #!!TODO

    flag += 1
    if flag >= 3:
        flag = 0
        
#     Auto ZOOM:
    


## !! need to implemnt confirmed target lost cannot be recovered
# #    emergency (search zoom):---v---
#         if notFound >= 5:
#             if notFound == 5:
#                 scale = 1
#                 centerX, centerY =  1640, 1232
#                 notFound += 1
#             elif notFound == 6:
#                 scale = 0.8
#                 notFound += 1
#             elif (notFound == 7) and (flag == 0):
#                 scale = 0.7
#                 notFound += 1
#             elif (notFound == 8) and (flag == 0):
#                 scale = 0.6
#                 scan(radiusX,radiusY)
#             elif notFound == 9:
#                 scale = 0.5
#                 scan(radiusX,radiusY)
#             elif notFound == 10:
#                 scale = 0.4
#                 scan(radiusX,radiusY)
#             elif notFound == 11:
#                 scale = 0.3
#                 scan(radiusX,radiusY)
#             elif notFound >= 12:
#                 notFound = 5
#                 print('done all!!!')
#     #             if CenterX >= Cwidth:
#     #                 notFound += 1


    #Boundary wall check:
    if centerX > 1640:
        if centerX + radiusX >= Cwidth:
            centerX = Cwidth - radiusX
    else:
        if centerX - radiusX < 0:
            centerX =  radiusX
            
    if centerY > 1232:
        if centerY + radiusY > Cheight:
            centerY = Cheight - radiusY
    else:
        if centerY - radiusY < 0:
            centerY =  radiusY
    
    
    
    minX,maxX=centerX-radiusX,centerX+radiusX
    minY,maxY=centerY-radiusY,centerY+radiusY
    
#     #0,0 = top right
    
    preimage = preimage[minY:maxY, minX:maxX]
#     preImage = preImage[minY:maxY, minX:maxX]
    #preImage = preImage[1000:2464 ,1000:3280]
    #                  [1:Y dir,1:X]
    
    #downscale for the AI

    
    
    
    
    outputSize = (WIDTH, HEIGHT)
    image = cv2.resize(preimage, outputSize)

    
    data = preprocess(image)
    cmap, paf = model_trt(data)
    cmap, paf = cmap.detach().cpu(), paf.detach().cpu()
    counts, objects, peaks = parse_objects(cmap, paf)#, cmap_threshold=0.15, link_threshold=0.15)
    #// start 
    
    
    for i in range(counts[0]):
#         print("Human index:%d "%( i ))
        get_keypoint(objects, i, peaks)
#     print("Human count:%d   (len:%d), notFound:%d "%(counts[0], len(counts), notFound))
#     if counts[0] == 0:
#         if notFound <= 4:
#             notFound += 1
#         if notFound > 1:
#             averageX = 0.5
#             averageY = 0.5
#             never gets here

    
#!!!!TODO
# average of (if none dont count!!!)
# nose, neck(between 2 eyes) (between 2 sholders) (ears dont cound its not allways visable ) 
# 50% mid point




    #// end
    
    
    

    
#     color1 = (255, 123, 0)
#     cv2.circle(image, (112, 112), 1, color1, 2, 1)
    
    
#    add 2nd screen
    draw_objects(image, counts, objects, peaks)

    image = cv2.copyMakeBorder(image,WIDTH,0,0,0,cv2.BORDER_CONSTANT,value=[10, 10, 10])
    
    
#     print("R-sholder: ", end='')
#     findAngle(peaks[0][8][0],peaks[0][6][0],peaks[0][17][0])
#     findAngle(peaks[0][10][0],peaks[0][8][0],peaks[0][6][0])
#     findAngle(peaks[0][17][0],peaks[0][5][0],peaks[0][7][0])
#     findAngle(peaks[0][5][0],peaks[0][7][0],peaks[0][9][0])

        
 
    
    
#     print(peaks[0][8][0][0] )
    if peaks[0][8][0][0] !=0 and peaks[0][6][0][0] !=0:
        R_sholder = peaks[0][8][0]-peaks[0][6][0]
    else: 
        R_sholder = [0,0]
    if peaks[0][10][0][0] !=0 and peaks[0][8][0][0] !=0 and R_sholder[0] != 0:
        R_elbow = peaks[0][10][0]-peaks[0][8][0]
    else: 
        R_elbow = [0,0]
    if peaks[0][7][0][0] !=0 and peaks[0][5][0][0] !=0:
        L_sholder = peaks[0][7][0]-peaks[0][5][0]
    else: 
        L_sholder = [0,0]
    if peaks[0][9][0][0] !=0 and peaks[0][7][0][0] !=0 and L_sholder[0] != 0:
        L_elbow = peaks[0][9][0]-peaks[0][7][0]
    else:
        L_elbow = [0,0]
        
    print("R-sholder: ", end='')    
    angle_R_sholder = findAngle(peaks[0][17][0],peaks[0][6][0],peaks[0][8][0])
    print(R_sholder,"\n")
    
    print("R-elbow  : ", end='')
    angle_R_elbow = findAngle(peaks[0][6][0],peaks[0][8][0],peaks[0][10][0])
    print(R_elbow,"\n")
    
    print("L-sholder: ", end='')
    angle_L_sholder = findAngle(peaks[0][7][0],peaks[0][5][0],peaks[0][17][0])
    print(L_sholder,"\n")
    
    print("L-elbow  : ", end='')
    angle_L_elbow = findAngle(peaks[0][9][0],peaks[0][7][0],peaks[0][5][0])
    print(L_elbow,"\n")
    print(" ")
    
    
    
    drwaw_stickman(image,R_sholder,R_elbow,L_sholder,L_elbow)
    
#   main pose estemeation
#     print(R_sholder)
    pose_word = poseEstemate(R_sholder,R_elbow,L_sholder,L_elbow,angle_R_sholder,angle_R_elbow,angle_L_sholder,angle_L_elbow)
    
    

    # Draw the text
    
    cv2.putText(image, pose_word[0], (0, 10), cv2.FONT_HERSHEY_SIMPLEX, (0.4), (36,255,12), 1)
    cv2.putText(image, pose_word[1], (140, 10), cv2.FONT_HERSHEY_SIMPLEX, (0.4), (36,255,12), 1)
    image = cv2.flip(image, 1)
    
    
    
#     drwaw_stickman(image,findAngle(peaks[0][17][0],peaks[0][6][0],peaks[0][8][0]),
#         findAngle(peaks[0][6][0],peaks[0][8][0],peaks[0][10][0]),
#         findAngle(peaks[0][7][0],peaks[0][5][0],peaks[0][17][0]),
#         findAngle(peaks[0][9][0],peaks[0][7][0],peaks[0][5][0]))
    
    
#   record video output
    if checkbox1.value == True:
        Rout.write(image)
        if time1() > end:
            camera.unobserve_all()
    
    image_w.value = bgr8_to_jpeg(image[:, ::-1, :])
    

    end = time.time()
    display("==== Net FPS :%f ===="%( 1 / (end - start)))

In [8]:
def poseEstemate(R_sholder,R_elbow,L_sholder,L_elbow,angle_R_sholder,angle_R_elbow,angle_L_sholder,angle_L_elbow):
    
    
    R_sholder = math.hypot(R_sholder[0],R_sholder[1])
    R_elbow   = math.hypot(R_elbow[0],R_elbow[1])
    L_sholder = math.hypot(L_sholder[0],L_sholder[1])
    L_elbow   = math.hypot(L_elbow[0],L_elbow[1])
    
    print(R_sholder)
    print(R_elbow)
    print(L_sholder)
    print(L_elbow)
    
    Right = "R_None"
    Left  = "L_None"
    

    if angle_R_sholder > 4:
        if angle_R_sholder < 90 :
            print("R_up")
            Right = "R_up"
        else:
            print("R_up_left")
            Right = "R_up_left"
            
    if angle_L_sholder > 4:
        if angle_L_sholder < 90 :
            print("L_up")
            Left = "L_up"
        else:
            print("L_up_right")
            Left = "L_up_right"
        
    if angle_R_sholder < -4:
        if angle_R_sholder > -90 :
            print("R_down")
            Right = "R_down"
        else:
            print("R_down_left")
            Right = "R_down_left"
    
    if angle_L_sholder < -4 :
        if angle_L_sholder > -90:
            print("L_down")
            Left = "L_down"
        else:
            print("L_down_right")
            Left = "L_down_right"
            
            
            
    if abs(angle_R_sholder) < 5 and abs(angle_R_elbow) < 8 and R_sholder != 0:
        print("R_straght")
        Right = "R_straght"
    if abs(angle_L_sholder) < 5 and abs(angle_L_elbow) < 8 and L_sholder != 0:
        print("L_straght")
        Left = "L_straght"
            
    return Right, Left
        
    
    

    
    
    
    
    
    
    
    
    
    
    
    
    return 0
    

In [9]:
# Pose estemation
import math

def findAngle(p1,p2,p3,inverse=0,verbose=1):
    
#     print(p1)
#     print(p2)
#     print(p3)

    x1, y1 = p1
    x2, y2 = p2
    x3, y3 = p3
    
    if x1 == 0 or x2 == 0 or x3 == 0:
        if verbose:
            print("0")
        return 0
    
#     angle = math.degrees(math.atan2(y1-y2,x1-x2) - math.atan2(y3-y2,x3-x2)) 
    angle = -math.degrees(math.atan2(y3-y2,x3-x2) - math.atan2(y1-y2,x1-x2))
    if inverse:
        angle -= angle
    if angle < 0:
        angle += 180
    else:
        angle -= 180

    if verbose:
        print(angle)

    return angle
#     except TypeError:
#         if verbose:
#             print("NOPE...")
#         return 0

    
    

In [10]:
# Draw stickman

def drwaw_stickman(image,R_sholderF,R_elbowF,L_sholderF,L_elbowF):
    color = (0, 255, 0)
    color1 = (255, 100, 255)
    color0 = (255, 238,0)
    
#     print(R_sholder,R_elbow,L_sholder,L_elbow)
    
    L_basepoint = [0.5,0.6]
    R_basepoint = [0.5,0.4]
    L_No = 0
    L_No = 0
    L_elbow_No = 0
    L_elbow_No = 0
    R_sholder,L_sholder,R_elbow,L_elbow = [0,0],[0,0],[0,0],[0,0]
    
    
#     R_sholder = angleToCord(R_basepoint,R_sholder)
#     L_sholder = angleToCord(R_basepoint,L_sholder)
    R_sholder[0], R_sholder[1]= R_basepoint[0]+R_sholderF[0],R_basepoint[1]+R_sholderF[1]
    L_sholder[0], L_sholder[1]= L_basepoint[0]+L_sholderF[0],L_basepoint[1]+L_sholderF[1]
    R_elbow[0]    , R_elbow[1]=   R_sholder[0]  +R_elbowF[0]  ,R_sholder[1]  +R_elbowF[1]
    L_elbow[0]    , L_elbow[1]=   L_sholder[0]  +L_elbowF[0]  ,L_sholder[1]  +L_elbowF[1]
    
#     if R_sholder[0]<=0:
        
#         if R_elbow[0] <= 0:
#             R_No = 1
#         else:
#             R_sholder[0],R_sholder[1]=  R_elbow[0]- (R_basepoint[0]-R_elbow[0])/2, R_elbow[1]- (R_basepoint[1]-R_elbow[1])/2
    
#     if R_elbow[0] <= 0:
#         R_elbow_No = 1
        
#     if L_sholder[0]<=0:
        
#         if L_elbow[0]<=  0:
#             L_No = 1
#         else:
#             L_sholder[0],L_sholder[1]=  L_elbow[0]- (L_basepoint[0]-L_elbow[0])/2, L_elbow[1]- (L_basepoint[1]-L_elbow[1])/2
    
#     if L_elbow[0]<=  0:
#         L_elbow_No = 1
            
#     edges = [R_basepoint,
#              R_sholder,
#              angleToCord(R_sholder,R_elbow),
#              L_basepoint,
#              L_sholder,
#              angleToCord(L_sholder,L_elbow)]
    edges = [R_basepoint,
                 R_sholder,
                 R_elbow,
                 L_basepoint,
                 L_sholder,
                 L_elbow]

    

    


    for peak in edges:
        if peak[0]>0 and peak[1]>0:
            if peak == (0.5,0.6) or peak == (0.5,0.4):
                color1 = (255, 100, 255)
            else:
                color0 = (255, 238,0)
            x = round(float(peak[1]) * WIDTH)
            y = round(float(peak[0]) * HEIGHT)
            cv2.circle(image, (x, y), 4, color0, 1)
        
        
      
        
    for i in range(0, 5):
        peak0 = edges[i]
        peak1 = edges[i+1]
        if peak0[0]>0 and peak1[0]>0 and peak0[1]>0 and peak1[1]>0:
            if i != 2:

                x0 = round(float(peak0[1]) * WIDTH)
                y0 = round(float(peak0[0]) * HEIGHT)
                x1 = round(float(peak1[1]) * WIDTH)
                y1 = round(float(peak1[0]) * HEIGHT)
                cv2.line(image, (x0, y0), (x1, y1), color, 1)

    
          
    
def angleToCord(P1,angle,length=0.2):
      
    y, x = P1
    angle += 180
    
    endy = y + length * math.sin(math.radians(angle))
    endx = x + length * math.cos(math.radians(angle))
    endP = endy,endx
    
    return(endP)
    

In [11]:
import ipywidgets
from IPython.display import display , clear_output


image_w = ipywidgets.Image(format='jpeg')
display(image_w)


button1 = ipywidgets.Button(description='Frame', button_style='success', layout={'width': '70px'})

def on_button1_clicked(_):
    with out:
        execute({'new': camera.value}) #get 1 frame 
button1.on_click(on_button1_clicked)


button2 = ipywidgets.Button(description='Stop', button_style='warning', layout={'width': '70px'})
def on_button2_clicked(_):
    camera.unobserve_all() #stop live feed
button2.on_click(on_button2_clicked)

            

button3 = ipywidgets.Button(description='Start', button_style='info', layout={'width': '70px'})
def on_button3_clicked(_):
    camera.observe(execute, names='value') #start live feed
button3.on_click(on_button3_clicked)


button4 = ipywidgets.Button(description='New file', button_style='info', layout={'width': '70px'})
def on_button4_clicked(_):
    Rout = cv2.VideoWriter(filename, VIDEO_TYPE['avi'], 10,  frameSize=(WIDTH, HEIGHT*2))
    end = time1() + 120
button4.on_click(on_button4_clicked)


button5 = ipywidgets.Button(description='End file', button_style='info', layout={'width': '70px'})
def on_button5_clicked(_):
    Rout.release() #save video
button5.on_click(on_button5_clicked)


out = ipywidgets.Output()

checkbox1 = ipywidgets.Checkbox(
    value=False,
    description='Record',
    indent=False,
    layout={'width': '70px'}
)


ipywidgets.VBox([
ipywidgets.HBox([button4,checkbox1,button5]),
ipywidgets.HBox([button1,button3,button2]),
out])


Image(value=b'', format='jpeg')

____

____

In [ ]:
execute({'new': camera.value}) #get 1 frame

In [89]:
camera.unobserve_all() #stop live feed

0:	nose            : success [0.420, 0.499]
1:	left_eye        : success [0.394, 0.533]
2:	right_eye       : success [0.411, 0.473]
3:	left_ear        : success [0.443, 0.609]
4:	right_ear       : success [0.463, 0.455]
5:	left_shoulder   : success [0.620, 0.689]
6:	right_shoulder  : success [0.619, 0.374]
7:	left_elbow      : success [0.810, 0.793]
8:	right_elbow     : success [0.827, 0.176]
17:	neck            : success [0.622, 0.533]
R-sholder: 
R-elbow  : 
L-sholder: 
L-elbow  : 
47.24874586240972
0
61.84815482079857
0


'==== Net FPS :10.631383 ===='

In [14]:
camera.observe(execute, names='value') #start live feed

In [ ]:
Rout.release() #save video

_______


Finally, we'll define the main execution loop.  This will perform the following steps

1.  Preprocess the camera image
2.  Execute the neural network
3.  Parse the objects from the neural network output
4.  Draw the objects onto the camera image
5.  Convert the image to JPEG format and stream to the display widget



In [1]:
# GPIO
import Jetson.GPIO as GPIO
GPIO.setmode(GPIO.BOARD) # mode with pin numbers


channels = [40, 38, 36]
GPIO.setup(channels, GPIO.OUT, initial=GPIO.LOW)



In [3]:
GPIO.output(channels, (GPIO.HIGH, GPIO.HIGH, GPIO.LOW))

In [5]:
GPIO.output(channels, (GPIO.LOW, GPIO.LOW, GPIO.LOW))

In [11]:
#python Test area:

top = 3.862
bot = 0.757

L_elbow =   math.hypot(top,bot)
print (L_elbow)




3.935491455968365


In [ ]:
#forgotten archive







# !sudo jt -t  monokai  -nfs 11 -fs 8 -cursw 3 -T -kl -N






# reload drawObjects:
# from importlib import reload


# import draw_objects
# reload(draw_objects)

# from draw_objects import DrawObjects1

# draw_objects = DrawObjects1(topology)